In [29]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,MaxPooling2D,Conv2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [3]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='sigmoid')(x) #final layer with softmax activation

c:\users\chira\appdata\local\programs\python\python37\lib\site-packages\keras_applications\mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'
W1007 19:48:56.730286 20740 deprecation_wrapper.py:119] From c:\users\chira\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1007 19:48:56.751230 20740 deprecation_wrapper.py:119] From c:\users\chira\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1007 19:48:56.775166 20740 deprecation_wrapper.py:119] From c:\users\chira\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Ple

In [25]:
base_model.output

<tf.Tensor 'conv_pw_13_relu/Relu6:0' shape=(?, ?, ?, 1024) dtype=float32>

In [33]:
base_model.input.__class__

tensorflow.python.framework.ops.Tensor

In [30]:
inp = Conv2D(256, (3, 3), input_shape=X.shape[1:])

In [32]:
from keras.layers import Input

In [ ]:
Input.

In [34]:
model=Model(inputs=base_model.input,outputs=preds)

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [36]:
l1 = model.layers[0]

In [37]:
l1.input.set_shape = (150,150,1)

In [24]:
l1.input_shape

(None, None, None, 3)

In [6]:
for layer in model.layers[:50]:
    layer.trainable=False
for layer in model.layers[50:]:
    layer.trainable=True
# or if we want to set the first 20 layers of the network to be non-trainable
# for layer in model.layers[:20]:
#     layer.trainable=False
# for layer in model.layers[20:]:
#     layer.trainable=True

In [7]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy']) 

W1007 19:52:10.005335 20740 deprecation_wrapper.py:119] From c:\users\chira\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1007 19:52:10.030268 20740 deprecation.py:323] From c:\users\chira\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
import pickle
import numpy as np
import pandas as pd
IMG_SIZE = 150

In [9]:
with open('Training_data.pkl','rb') as file:
    Training_data = pickle.load(file)

In [10]:
X = []
y = []
for feature , target in Training_data:
    X.append(feature)
    y.append(target)

In [11]:
X = np.array(X).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0, random_state=42)

In [14]:
model.fit(X_train, [y_train], epochs = 3, batch_size = 32,verbose=1, validation_split=0.0, validation_data=None, shuffle=True)

ValueError: Error when checking input: expected input_1 to have shape (None, None, 3) but got array with shape (150, 150, 1)

In [ ]:
TEST_DATA_DIR = "DataSet/TestImages"
TestDataPath = os.path.join(TEST_DATA_DIR)
Testing_data = []
def create_Testing_data():
    for image in tqdm(os.listdir(TestDataPath)):
        img_array = cv2.resize(cv2.imread(os.path.join(TestDataPath,image),cv2.IMREAD_GRAYSCALE) ,(IMG_SIZE,IMG_SIZE))
        Testing_data.append([img_array,image])
create_Testing_data()
imageFileNames = []
testImageData = []
for item in Testing_data:
    testImageData.append(item[0])
    imageFileNames.append(item[1])
testImageData = np.array(testImageData).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [ ]:
import time
modelFileName = "TrainedTransferLearningModelOldData{}.h5".format(int(time.time()))
model.save(modelFileName)

In [ ]:
# model.evaluate_generator(generator=train_generator,steps = nb_samples)

In [ ]:
outPredictions = model.predict(testImageData)

In [ ]:
class_Output = [1 if item[0]>0.5 else 0 for item in outPredictions]

In [ ]:
import time
data = {'Image_File': imageFileNames,'Class':class_Output}
predictionDataFrame = pd.DataFrame(data)
outfilename = "Prediction{}.csv".format(int(time.time()))
predictionDataFrame.to_csv(outfilename, index=False)